In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy.linalg as la
import scipy.linalg as sci

In [ ]:
# importing our data sets
guesses = pd.read_csv("valid_guesses.csv")
solutions = pd.read_csv("valid_solutions.csv")

In [ ]:
# initiate a letter dictionary
letter_dict = {"a":1,"b":2,"c":3, "d":4, "e":5, "f":6, "g":7, "h":8, "i":9, "j":10, "k":11, "l":12, "m":13, "n":14,
               "o": 15, "p":16, "q":17, "r":18, "s":19, "t":20, "u":21, "v":22, "w":23, "x":24, "y":25, "z":26}
# convert all the guesses words to a matrix as stated in the paper
out = []
for i in guesses["word"]:
  letters = [*i]
  out2 = []
  for j in letters:
    out2.append(letter_dict[j])
  out.append(out2)
colg_letters = []
for i in out:
  word = []
  for l in i:
    for j in range(26):
      if l == j+1:
        word.append(1)
      else:
        word.append(0)
  colg_letters.append(word)
#end with a matrix of all the guessing words repeat the same for the solution words
out = []
for i in solutions["word"]:
  letters = [*i]
  out2 = []
  for j in letters:
    out2.append(letter_dict[j])
  out.append(out2)
cols_letters = []
for i in out:
  word = []
  for l in i:
    for j in range(26):
      if l == j+1:
        word.append(1)
      else:
        word.append(0)
  cols_letters.append(word)

In [ ]:
# create the all word matrix by combining the guesse and solution columns
# some casting to numpy arrays and matrices to make computation quicker
all_words = colg_letters + cols_letters
all_words = np.transpose(np.asmatrix(np.array([np.array(xi) for xi in all_words])))
guess= np.transpose(np.asmatrix(np.array([np.array(xi) for xi in colg_letters])))

In [ ]:
def words_to_vectors(words):
  """
  words: a list of words
  return: a matrix of vectors for the words
  """
  # same code as above in a function used in the future to quickly turn remainign solution space into a matrix
  out = []
  for i in words:
    letters = [*i]
    out2 = []
    for j in letters:
      out2.append(letter_dict[j])
    out.append(out2)
  colg_letters = []
  for i in out:
    word = []
    for l in i:
      for j in range(26):
        if l == j+1:
          word.append(1)
        else:
          word.append(0)
    colg_letters.append(word)
    solution= np.transpose(np.asmatrix(np.array([np.array(xi) for xi in colg_letters])))
  return solution

In [ ]:
def vector_to_word(vector):
  """
  vector; takes in a word vector
  output: the word vector converted back into a word
  """
  # get the indexes of the letters
  vector1 = (vector).tolist()
  letter1 = vector1[0][0:26]
  letter2 = vector1[0][26:52]
  letter3 = vector1[0][52:78]
  letter4 = vector1[0][78:104]
  letter5 = vector1[0][104:130]
  word = ""
  # enumerate over each letter and find its corresponding letter based on dictionary of letters above
  # constructs the word from each letter
  for i,j in enumerate(letter1):
    if j == 1:
      word = list(letter_dict.keys())[i]
  for i,j in enumerate(letter2):
    if j == 1:
      word = word + list(letter_dict.keys())[i]
  for i,j in enumerate(letter3):
    if j == 1:
      word = word + list(letter_dict.keys())[i]
  for i,j in enumerate(letter4):
    if j == 1:
      word = word + list(letter_dict.keys())[i]
  for i,j in enumerate(letter5):
    if j == 1:
      word = word + list(letter_dict.keys())[i]
  return word

In [ ]:
def best_word(solution, all_words):
  """
  solution: remaining solution space
  all_words: the whole action space
  """
  #Calculate AA^T and the associated eigenvectors
  AAt = np.matmul(solution, np.transpose(solution))
  w,v = la.eigh(AAt)
  #Select the msot dominant eigenvector and norm of the eigenvector
  dom1 = np.abs(v[:,-1])
  normdom = np.linalg.norm(dom1)
  #initiate maximum angle and empty word out
  word = ""
  angle_min = 90
  #Loop over all words psosible playable words find cosine simmilairty and determine if cosines similarity is smaller than
  # the smallest iterated word store angle_min as the new angle and the word as the new word
  for i in range(0,all_words.shape[1]):
    angle1 = np.degrees(np.arccos(np.matmul(np.transpose(all_words[:,i]),dom1)/(np.linalg.norm(all_words[:,i]) * normdom)))
    if np.abs(angle1) < angle_min:
          angle_min = angle1
          word = all_words[:,i]
  #convert vectorized word into normal word
  return vector_to_word(np.transpose(word))

In [ ]:
# we see the best starting word for rank-1 approximations is "saine"
best_word(words_to_vectors(solutions["word"]), all_words)

(130, 1)


'saine'

In [ ]:
# https://www.practicepython.org/blog/2022/02/12/wordle.html
#code adapted to fit our code from the website above, just borrowed to save time in writting compare function
# verified code worked to our specifications
def find_all_char_positions(word: str, char: str):
    positions = []
    pos = word.find(char)
    while pos != -1:
        positions.append(pos)
        pos = word.find(char, pos + 1)
    return positions
# https://www.practicepython.org/blog/2022/02/12/wordle.html
#code adapted to fit our code from the website above, just borrowed to save time in writting compare function
# verified code worked to our specifications
def compare(expected: str, guess: str) :
    """
    Expected : is the secret word
    guess : is our guessed word
    output: list of length 5 and elements in 0,1,2 where 2 is equivalent to green, 1 is equivalent to yellow , 0 is equivalent to gray
    """
    output = [0] * len(expected)
    counted_pos = set()
    for index, (expected_char, guess_char) in enumerate(zip(expected, guess)):
        if expected_char == guess_char:
            output[index] = 2
            counted_pos.add(index)
    for index, guess_char in enumerate(guess):
        if guess_char in expected and \
                output[index] != 2:
            positions = find_all_char_positions(word=expected, char=guess_char)
            for pos in positions:
                if pos not in counted_pos:
                    output[index] = 1
                    counted_pos.add(pos)
                    break
    return output

In [ ]:
# garbage code that we did not use
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)
def update_solution(guess, key, solutions):
  skips2 = []
  skips3 = []
  for i in guess:
    if len(list(find_all(guess, i))) == 2:
      skips2 = [list(find_all(guess, i))[1]]
    elif len(list(find_all(guess, i))) == 3:
      skips3 = [list(find_all(guess, i))[2]]
  solutions = solutions
  Solution = solutions
  print(skips2)
  print(skips3)
  for i, j in enumerate(key):
    if i not in skips2 and i not in skips3:
      if j == 0:
        for k in Solution:
          if guess[i] in k:
            if k in solutions:
              solutions.remove(k)
      elif j == 1:
        for k in Solution:
          if guess[i] not in k:
            if k in solutions:
              solutions.remove(k)
          if k[i] == guess[i]:
              if k in solutions:
                solutions.remove(k)
      elif j == 2:
        for k in Solution:
          if guess[i] != k[i]:
            if k in solutions:
              solutions.remove(k)
    if i in skips2:
      if j ==0:
        for k in solutions:
          if len(list(find_all(k, guess[i]))) > 1:
            if k in solutions:
              solutions.remove(k)
      elif j == 2:
        for k in Solution:
          if len(list(find_all(k, guess[i]))) < 2:
            if k in solutions:
              solutions.remove(k)
          if guess[i] != k[i]:
            if k in solutions:
              solutions.remove(k)
      elif j ==1:
        for k in Solution:
          if len(list(find_all(k, guess[i]))) < 2:
            if k in solutions:
              solutions.remove(k)
          if guess[i] == k[i]:
            if k in solutions:
              solutions.remove(k)
      elif i in skips3:
        if j == 0:
          for k in solutions:
            if len(list(find_all(k, guess[i]))) > 2:
              if k in solutions:
                solutions.remove(k)
        if j == 2:
          for k in Solution:
            if len(list(find_all(k, guess[i]))) < 3:
              if k in solutions:
                solutions.remove(k)
            if guess[i] != k[i]:
              if k in solutions:
                solutions.remove(k)
        if j ==1:
          for k in Solution:
            if len(list(find_all(k, guess[i]))) < 3:
              if k in solutions:
                solutions.remove(k)
            if guess[i] == k[i]:
              if k in solutions:
                solutions.remove(k)
  return solutions

In [ ]:
def update_solution( g, colors,S):
    """Given a current solution list, a guess, and the coloring that comes up
    when we make that guess, returns the new list of possible solutions after
    applying all of the new information we learn from the coloring of the guess."""
    if g in S:
      S.remove(g)

    j = 0
    while j < len(S): # iterating through the remaining words to see which ones get eliminated
      keep = True
      for i in range(len(S[j])):
        keep = keep and not (colors[i] == 2 and S[j][i] != g[i])
        keep = keep and not (colors[i] == 1 and g[i] not in S[j])
        keep = keep and not (colors[i] == 1 and S[j][i] == g[i])
        keep = keep and not (colors[i] == 1 and S[j].count(g[i]) < g[:i+1].count(g[i]))
        keep = keep and not (colors[i] == 0 and S[j].count(g[i]) >= g.count(g[i]))
      if keep:
          j += 1
      else:
          S.pop(j)
    return S

In [ ]:
#testing to make sure our update solution space function works
solutions = ['happy', "nanny", "siane", "slips", "strip", "mommy", "booby", "pickr"]
update_solution("fanny", compare("booby", "fanny"), solutions)

['mommy', 'booby']

In [ ]:
solutions = list(pd.read_csv("valid_solutions.csv")["word"])
# code that iterates through a game of wordle
def solve_wordle(secret, solutions, all_words):
  # round 1 assume the starting word is saine to save computation time
  #updates the solution space absed on the coloring
  solutions = update_solution("saine", compare(secret,"saine"), solutions)
  if len(solutions) == 1:
    return 1, solutions[0], 2
  solutions1 = solutions
  #turns remaining solution space into vectors
  solutions = words_to_vectors(solutions)
  # find the best word
  out1 = best_word(solutions,all_words)
  if out1 == secret:
    return 1, out1,2
  #we repeat for each turn
  # ROund 2
  solutions = update_solution(out1, compare(secret,out1), solutions1)
  if len(solutions) == 1:
    return 1, solutions[0],3
  solutions1 = solutions
  solutions = words_to_vectors(solutions)
  out1 = best_word(solutions,all_words)
  if out1 == secret:
    return 1, out1,3
  # Round 3
  solutions = update_solution(out1, compare(secret,out1), solutions1)
  if len(solutions) == 1:
    return 1, solutions[0], 4
  solutions1 = solutions
  solutions = words_to_vectors(solutions)
  out1 = best_word(solutions,all_words)
  if out1 == secret:
    return 1, out1, 4
  # Round 4
  solutions = update_solution(out1, compare(secret,out1), solutions1)
  if len(solutions) == 1:
    return 1, solutions[0], 5
  solutions1 = solutions
  solutions = words_to_vectors(solutions)
  out1 = best_word(solutions,all_words)
  if out1 == secret:
    return 1, out1, 5
  # Round 5
  solutions = update_solution(out1, compare(secret,out1), solutions1)
  if len(solutions) == 1:
    return 1, solutions[0],6
  solutions1 = solutions
  solutions = words_to_vectors(solutions)
  out1 = best_word(solutions,all_words)
  if out1 == secret:
    return 1, out1,6
  else:
    return 0, 0 ,7


In [ ]:
from random import sample

In [ ]:
import sklearn.metrics as skm

In [ ]:
# best word again but using PCA and cosine similairty
def best_word(solution, all_words):
  # do the SVD
  U, S, Vh = np.linalg.svd(solution, full_matrices=True)
  if len(S)>=3:
    # Calculate the three best rank 1 approximations
    dom1 = np.matmul(U[:,0], Vh[0])*S[0]
    dom2 = np.matmul(U[:,1], Vh[1])*S[1]
    dom3 = np.matmul(U[:,2], Vh[2])*S[2]


    #commented out code that is used for producing plots but not used in best word algortihm
    #dom11 = np.matmul(U[:,3], Vh[3])*S[3]
    #dom21 = np.matmul(U[:,4], Vh[4])*S[4]
    #dom31 = np.matmul(U[:,5], Vh[5])*S[5]
    #dom12 = np.matmul(U[:,6], Vh[6])*S[6]
    #dom22 = np.matmul(U[:,7], Vh[7])*S[7]
    #dom32 = np.matmul(U[:,8], Vh[8])*S[8]


    #Calculating the percent explained
    percent1 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom1))
    percent2 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom2))
    percent3 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom3))
    #commented out code that is used for producing plots but not used in best word algortihm
    #percent11 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom11))
    #percent21 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom21))
    #percent31 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom31))
    #percent12 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom12))
    #percent22 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom22))
    #percent32 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom32))
    # calculating the norm of the rank 1
    normdom1 = np.linalg.norm(dom1[:,0])
    normdom2 = np.linalg.norm(dom2[:,0])
    normdom3 = np.linalg.norm(dom3[:,0])

    #commented out code that is used for producing plots but not used in best word algortihm
    #plt.plot(np.arange(1,10), [percent1, percent2, percent3, percent11, percent21, percent31, percent12, percent22, percent32])
    #plt.plot(np.arange(1,4), [percent1, percent2, percent3])
    #plt.title('Principle components Vs Percent Variance Explained')
    #plt.xlabel('Principle Component')
    #plt.ylabel('Percent Variance Explained')
    #plt.show()

    word = ""
    angle_min = 0
    for i in range(0,all_words.shape[1]):
      angle1 = np.degrees(np.arccos(np.matmul(np.transpose(dom1[:,0]), (all_words[:,i]))/(np.linalg.norm(all_words[:,i]) * normdom1)))
      angle2 = np.degrees(np.arccos(np.matmul(np.transpose(dom2[:,0]), (all_words[:,i]) )/(np.linalg.norm(all_words[:,i]) * normdom2)))
      angle3 = np.degrees(np.arccos(np.matmul(np.transpose(dom3[:,0]), (all_words[:,i]) )/(np.linalg.norm(all_words[:,i]) * normdom3)))
      s1 = percent1 * (90 -angle1)
      s2 = percent2 * (90 -angle2)
      s3 = percent3 * (90 -angle3)
      if s1 > angle_min:
            angle_min = s1
            word = all_words[:,i]
      if s2 > angle_min:
            angle_min = s2
            word = all_words[:,i]
      if s3 > angle_min:
            angle_min = s3
            word = all_words[:,i]
  else:
    # repeated code if we only have 2 solutions remaining
    dom1 = np.matmul(U[:,0], Vh[0])*S[0]
    dom2 = np.matmul(U[:,1], Vh[1])*S[1]
    percent1 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom1))
    percent2 = skm.explained_variance_score(np.asarray(solution), np.asarray(dom2))
    normdom1 = np.linalg.norm(dom1[:,0])
    normdom2 = np.linalg.norm(dom2[:,0])
    word = ""
    angle_vector = np.array([])
    angle_min = 0
    count = 0
    for i in range(0,all_words.shape[1]):
      angle1 = np.degrees(np.arccos(np.matmul(np.transpose(dom1[:,0]), (all_words[:,i]))/(np.linalg.norm(all_words[:,i]) * normdom1)))
      angle2 = np.degrees(np.arccos(np.matmul(np.transpose(dom2[:,0]), (all_words[:,i]) )/(np.linalg.norm(all_words[:,i]) * normdom2)))
      count += 1
      s1 = percent1 * (90 -angle1)
      s2 = percent2 * (90 -angle2)
      if s1 > angle_min:
            angle_min = s1
            word = all_words[:,i]
      if s2 > angle_min:
            angle_min = s2
            word = all_words[:,i]
  return vector_to_word(np.transpose(word))

In [ ]:
# finding the best first word using the new strategy we see saine is the best
#word again we can use the same solve_wordle function from above
solution = list(pd.read_csv("valid_solutions.csv")["word"])
solution = words_to_vectors(solution)
best_word(solution, all_words)

'saine'

In [ ]:
# testing our code on multiple examples
import random as rand
secrets = list(pd.read_csv("valid_solutions.csv")["word"])
attempts = []
solved = []
Sk = rand.choices(secrets, k=10)
for i in secrets[0:100]:
  solutions = list(pd.read_csv("valid_solutions.csv")["word"])
  x = solve_wordle(i, solutions, all_words)
  solved.append(x[0])
  if solved[-1]:
    attempts.append(x[2])
print(sum(solved))
print(sum(solved)/len(solved))
print(sum(attempts)/len(attempts))

<ipython-input-50-43eb6ed85b75>:49: RuntimeWarning: invalid value encountered in divide
  angle2 = np.degrees(np.arccos(np.matmul(np.transpose(dom2[:,0]), (all_words[:,i]) )/(np.linalg.norm(all_words[:,i]) * normdom2)))
<ipython-input-50-43eb6ed85b75>:50: RuntimeWarning: invalid value encountered in divide
  angle3 = np.degrees(np.arccos(np.matmul(np.transpose(dom3[:,0]), (all_words[:,i]) )/(np.linalg.norm(all_words[:,i]) * normdom3)))
<ipython-input-50-43eb6ed85b75>:49: RuntimeWarning: invalid value encountered in divide
  angle2 = np.degrees(np.arccos(np.matmul(np.transpose(dom2[:,0]), (all_words[:,i]) )/(np.linalg.norm(all_words[:,i]) * normdom2)))
<ipython-input-50-43eb6ed85b75>:50: RuntimeWarning: invalid value encountered in divide
  angle3 = np.degrees(np.arccos(np.matmul(np.transpose(dom3[:,0]), (all_words[:,i]) )/(np.linalg.norm(all_words[:,i]) * normdom3)))
<ipython-input-50-43eb6ed85b75>:50: RuntimeWarning: invalid value encountered in divide
  angle3 = np.degrees(np.arccos(

100
1.0
3.36


In [ ]:
np.mean(attempts)

3.36

In [ ]:
np.sum(correct)

NameError: name 'correct' is not defined